<a href="https://colab.research.google.com/github/juyounLee/DeepLearningZeroToAll_TF2./blob/main/06_02_softmax_animal_classfier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import tensorflow as tf
import numpy as np
import math

In [46]:
# load the data
xy = np.loadtxt('/content/data-04-zoo.csv', delimiter= ',', dtype=np.float32)
x_data = xy[:, :-1]
y_data = xy[:, [-1]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.int32)

In [47]:
# make y_data as one_hot shape
y_data = tf.one_hot(list(y_data), 7) 
y_data = tf.reshape(y_data, [-1, 7])

In [48]:
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

In [49]:
print(x_data.shape)
print(y_data.shape)

(101, 16)
(101, 7)


In [50]:
# initialize the variables
W = tf.Variable(tf.random.normal([16, 7]))
b = tf.Variable(tf.random.normal([7]))

print(W.numpy())

[[-0.02024945  2.2436645  -0.3504013   0.03615203 -0.42295384 -0.05372311
  -2.899066  ]
 [-0.39605343  0.43462005  1.3968053   2.8523948   0.30932766 -0.6993349
  -0.16902241]
 [-0.2076432  -1.1195499   0.04066771  1.1970735   1.1558977   0.90011865
  -1.409623  ]
 [ 3.658415    0.46351415 -0.9704024   0.31088263 -0.7054393  -0.42168263
   0.6267286 ]
 [ 0.10103312  0.23256825  1.4911426   0.00673932 -0.5886298   1.4188979
   1.1518407 ]
 [ 0.28439945 -1.3698899  -0.45730296 -1.6179603   1.5069158   1.5320723
   0.32169214]
 [-0.23315333 -1.2260298  -0.20720561 -1.0834594  -1.4734671  -0.46172002
   0.6614839 ]
 [-0.6336018   1.0821158  -0.6146318   0.99988794  0.4701772   1.6525385
  -0.8558768 ]
 [ 1.5001061  -0.2242447   1.949952   -2.126988   -0.44817838 -0.13736278
   0.6110684 ]
 [ 0.61475396 -0.14767304  0.5289491  -1.3826345  -0.78454745 -1.1588423
   1.4229065 ]
 [ 0.8049735  -0.5146065  -1.2408112   1.6487114   1.6021543  -0.7977655
  -0.6400068 ]
 [-0.4990754  -1.010472   -

In [51]:
# logit function
def logit_fn(x):
  logit = tf.matmul(x, W)+b
  return logit

In [52]:
# hypothesis
def hypothesis_fn(x):
  logit = logit_fn(x)
  hypothesis = tf.nn.softmax(logit)
  return hypothesis

In [53]:
# cost function
def cost_fn(x, y):
  hypothesis = hypothesis_fn(x)
  logit = logit_fn(x)
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, logits=logit))
  return cost

In [54]:
# calculate the gradient
def grad(x, y):
  with tf.GradientTape() as tape:
    loss_value = cost_fn(x, y)
  return tape.gradient(loss_value, [W, b])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [57]:
for step in range(2001):
  for feature, label in iter(dataset):
    grads = grad(feature, label)
    optimizer.apply_gradients(grads_and_vars=zip(grads, [W, b]))
    cost = cost_fn(feature, label)

    if step%200 == 0:
      print('step: {} cost: {:.4f}'.format(step, cost.numpy()))

step: 0 cost: 3.6286
step: 200 cost: 1.4340
step: 400 cost: 0.9909
step: 600 cost: 0.7838
step: 800 cost: 0.6643
step: 1000 cost: 0.5840
step: 1200 cost: 0.5240
step: 1400 cost: 0.4762
step: 1600 cost: 0.4366
step: 1800 cost: 0.4029
step: 2000 cost: 0.3740


In [59]:
print(x_data[3])
print(y_data[3])

[1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 4. 0. 0. 1.]
tf.Tensor([1. 0. 0. 0. 0. 0. 0.], shape=(7,), dtype=float32)


In [69]:
x_data = [[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.]]
x_data = np.array(x_data, dtype=np.float32)

y_estimate = hypothesis_fn(x_data)
print(tf.argmax(y_estimate, 1))

tf.Tensor([0], shape=(1,), dtype=int64)
